In [ ]:
import tess_cpm
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS

In [ ]:
fits_file = "supernovae/tess-s0001-4-1_64.525833_-63.615669_100x100_astrocut.fits"

In [ ]:
sn = tess_cpm.CPM(fits_file, remove_bad=True)

In [ ]:
sn.set_poly_model(1, 4, 2)
sn.set_target(50, 50)
sn.set_exclusion(10)
sn.set_predictor_pixels(128*1)

In [ ]:
%%timeit
sn.lsq(0.1, rescale=True, polynomials=True)

In [ ]:
tess_cpm.summary_plot(sn, 20, subtract_polynomials=False, save=False)

In [ ]:
fits_file = "supernovae/tess-s0002-4-1_64.525833_-63.615669_100x100_astrocut.fits"

In [ ]:
sector2 = tess_cpm.CPM(fits_file, remove_bad=False)

In [ ]:
sector2.set_poly_model(2, 4, 0.5)
sector2.set_target(50, 50)
sector2.set_exclusion(10)
sector2.set_predictor_pixels(256)

In [ ]:
sector2.lsq(0.1, rescale=True, polynomials=True)

In [ ]:
tess_cpm.summary_plot(sector2, 20, subtract_polynomials=False, save=False)